First, some setup code, so we can run this on Colab. No need to run this on a local machine.

In [2]:
!git clone https://github.com/smeenehan/TensorFlowProjects.git

fatal: destination path 'TensorFlowProjects' already exists and is not an empty directory.
/content/TensorFlowProjects/CIFAR


In [3]:
%cd TensorFlowProjects/CIFAR

/content/TensorFlowProjects/CIFAR


In [2]:
from model import ResNet, res_net_layers
import tensorflow as tf
import tensorflow.contrib.eager as tfe
from train import ModelTrainer, data_spec
import utils
%matplotlib inline

c:\users\smmeenehan\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [0]:
eager_mode = False
if eager_mode:
    tf.enable_eager_execution()

In [7]:
tf.reset_default_graph()

In [8]:
device, data_format = utils.device_and_data_format()
train_data, val_data, test_data, x_test_raw = utils.get_CIFAR10_data(data_format)

In [9]:
model = ResNet(data_format)#res_net_layers(data_format)
loss = tf.losses.sparse_softmax_cross_entropy
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)

In [10]:
trainer = ModelTrainer(model, loss, optimizer, data_spec=data_spec(train_data))

In [11]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    trainer.train(val_data, val_data=val_data, num_epochs=20)

Step #0 	Loss: 19.857330

Train time for epoch #1: 18.743000

Validation loss for epoch #1: 96.326969
Step #0 	Loss: 7.799016

Train time for epoch #2: 18.441000

Validation loss for epoch #2: 9.153396
Step #0 	Loss: 2.338305

Train time for epoch #3: 17.270000

Validation loss for epoch #3: 3.774281
Step #0 	Loss: 1.977716

Train time for epoch #4: 17.320000

Validation loss for epoch #4: 2.604521
Step #0 	Loss: 1.553020

Train time for epoch #5: 18.453000

Validation loss for epoch #5: 1.802825
Step #0 	Loss: 1.253288

Train time for epoch #6: 18.265000

Validation loss for epoch #6: 1.435306
Step #0 	Loss: 1.085570

Train time for epoch #7: 18.875000

Validation loss for epoch #7: 1.235994
Step #0 	Loss: 0.913218

Train time for epoch #8: 18.239000

Validation loss for epoch #8: 1.514332
Step #0 	Loss: 0.933635

Train time for epoch #9: 18.577000

Validation loss for epoch #9: 1.283576
Step #0 	Loss: 0.703749

Train time for epoch #10: 17.023000

Validation loss for epoch #10: 1.595

KeyboardInterrupt: 